# Set up PyTorch and Triton Containers

This step will download and run two docker containers from NGC. We will use the PyTorch container to build a predictive model using XGBoost. Then we will deploy that model to the Triton container. The two containers are networked so that requests from the PyTorch container can be submitted to the Triton server.

### Set up notes

* Make sure you have properly set up your server as described in [Configuring the server]().
* You should have plenty of disk space (at least 64 GB)
* Open parts 8000, 8001, 8002 for Triton
* Open port 8888 for Jupyter Lab

## Set up docker network

Create a docker network for the containers to communicate with each other.

In [ ]:
sudo docker network create tritonnet

## Pull and run docker containers

Pull the contains are run them within the network. Mount a volume on the PyTorch container. This volume will be used to save models that will later be transfered to the Triton model repository.

In [ ]:
# Triton
sudo docker pull nvcr.io/nvidia/tritonserver:22.03-py3
sudo docker run --gpus=all -d -p 8000:8000 -p 8001:8001 -p 8002:8002 --network=tritonnet -v /home/azureuser/model_repository:/models \
    --name tritonserver nvcr.io/nvidia/tritonserver:22.03-py3 tritonserver --model-repository=/models

# PyTorch
sudo docker volume create volume1
sudo docker pull nvcr.io/nvidia/pytorch:22.03-py3
sudo docker run --gpus=all -t -d -p 8888:8888 --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 --network=tritonnet \
    --mount source=volume1,destination=/workspace/triton --name pytorch nvcr.io/nvidia/pytorch:22.03-py3

## Start Jupyter Lab

Jupyter Lab is preinstalled on the PyTorch container. You can run it in headless mode on port 8888. Note that the code below removes the token. If you want to use a security token when you log into the server, you can remove the comment `NotebeookApp.token=''`. Once Jupyter Lab is running, you can access it at `http://<server-ip>:8888/lab?`.

In [ ]:
sudo docker exec -it pytorch /bin/bash
nohup jupyter-lab --NotebookApp.token='' --no-browser --port=8888 &
exit

## Identify Triton IP

Make a note of the IP of the Triton server for the later on.

In [ ]:
sudo docker network inspect tritonnet

In [ ]:
sudo docker logs tritonserver